In [1]:
# import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#from sklearn.model_selection import train_test_split

# Add your import statements here
from keras import Input, Model
from keras.layers import Activation, TimeDistributed, Dense, RepeatVector, Embedding, multiply
from keras.layers.recurrent import LSTM, GRU
from keras.callbacks import TensorBoard, EarlyStopping
from keras.optimizers import Adam, RMSprop
#from keras.models import Sequential
from keras.layers import merge
from keras.layers.core import *
from keras.models import *

#from attention_utils import get_activations, get_data_recurrent

import pickle

Using TensorFlow backend.


## Fetch even data

In [2]:
with open('X_train_even.pickle', 'rb') as handle:
    X_train_even = pickle.load(handle)
    
# with open('X_test_even.pickle', 'rb') as handle:
#     X_test_even = pickle.load(handle)
    
with open('Y_train_even.pickle', 'rb') as handle:
    Y_train_even = pickle.load(handle)
    
# with open('Y_test_even.pickle', 'rb') as handle:
#     Y_test_even = pickle.load(handle)

print('Total number of even training data points: %d' %len(X_train_even))

Total number of even training data points: 199483


## First model on even data

In [3]:
def create_model(X_seq_len, output_n, hidden_dim, vocabulary_size, embedding_dim):

    model = Sequential()
    model.add(Embedding(vocabulary_size, embedding_dim, input_length=X_seq_len))
    model.add(LSTM(hidden_dim))
    model.add(Dense(output_n,activation='sigmoid'))
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

#### Create a smaller data set for testing

In [4]:
lesser_number = 10000

small_Y = Y_train_even[0:lesser_number]
small_X = X_train_even[0:lesser_number]
neg_share = len(small_Y[small_Y==1])/lesser_number

print('The share of negative reviews:')
print(neg_share)
print('It is approximately even!')

The share of negative reviews:
0.4924
It is approximately even!


#### Set the data to train for 

In [5]:
X_train_here = X_train_even
Y_train_here = Y_train_even

#### Flatten input data for embedding layer

In [6]:
dim1 = len(X_train_here)
dim2 = len(X_train_here[0])

X_train_here_flat = X_train_here.reshape(dim1,dim2)

### Train!

In [18]:
X_seq_len = len(X_train_here[0]) #the input data is padded
output_dim = 1
hidden_dim = 10
num_epochs = 10
b_size = 64

vocabulary_size = 46210+1 #from Data Processing3.0
embedding_dim = 10

model = create_model(X_seq_len,output_dim,hidden_dim, vocabulary_size, embedding_dim)
name = 'second_RNN_even_data_more'

# Define a tensorboard callback
tb = TensorBoard(log_dir='./logs/'+name)
    
# Print model summary and train
model.summary()
current_history = model.fit(X_train_here_flat, Y_train_here, verbose=1, batch_size=b_size, epochs=num_epochs, callbacks=[tb], validation_split=0.2);
    

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 100, 10)           462110    
_________________________________________________________________
lstm_6 (LSTM)                (None, 10)                840       
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 11        
Total params: 462,961
Trainable params: 462,961
Non-trainable params: 0
_________________________________________________________________
Train on 159586 samples, validate on 39897 samples
Epoch 1/10
 16064/159586 [==>...........................] - ETA: 12:25 - loss: 0.6856 - acc: 0.5361

KeyboardInterrupt: 

### Second-second model Train!

In [ ]:
X_seq_len = len(X_train_here[0]) #the input data is padded
output_dim = 1
hidden_dim = 50
num_epochs = 10
b_size = 64

vocabulary_size = 46210+1 #from Data Processing3.0
embedding_dim = 20

model = create_model(X_seq_len,output_dim,hidden_dim, vocabulary_size, embedding_dim)
name = 'second_RNN_more_param'

# Define a tensorboard callback
tb = TensorBoard(log_dir='./logs/'+name)
    
# Print model summary and train
model.summary()
current_history = model.fit(X_train_here_flat, Y_train_here, verbose=1, batch_size=b_size, epochs=num_epochs, callbacks=[tb], validation_split=0.2);

### Second-third model Train!

In [7]:
X_seq_len = len(X_train_here[0]) #the input data is padded
output_dim = 1
hidden_dim = 50
num_epochs = 10
b_size = 64

vocabulary_size = 46210+1 #from Data Processing3.0
embedding_dim = 200

model = create_model(X_seq_len,output_dim,hidden_dim, vocabulary_size, embedding_dim)
name = 'second_RNN_even_data_more_embedding'

# Define a tensorboard callback
tb = TensorBoard(log_dir='./logs/'+name)
    
# Print model summary and train
model.summary()
current_history = model.fit(X_train_here_flat, Y_train_here, verbose=1, batch_size=b_size, epochs=num_epochs, callbacks=[tb], validation_split=0.2);

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 200)          9242200   
_________________________________________________________________
lstm_1 (LSTM)                (None, 50)                50200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 51        
Total params: 9,292,451
Trainable params: 9,292,451
Non-trainable params: 0
_________________________________________________________________
Train on 159586 samples, validate on 39897 samples
Epoch 1/10
159586/159586 [==============================] - 624s 4ms/step - loss: 0.4391 - acc: 0.7805 - val_loss: 0.3021 - val_acc: 0.8749
Epoch 2/10
159586/159586 [==============================] - 617s 4ms/step - loss: 0.2474 - acc: 0.9019 - val_loss: 0.2689 - val_acc: 0.8901
Epoch 3/10
159586/159586 [==============================] - 613s 

KeyboardInterrupt: 

### Accidental overwrite more_embedding with larger hidden_dim

In [8]:
X_seq_len = len(X_train_here[0]) #the input data is padded
output_dim = 1
hidden_dim = 100
num_epochs = 10
b_size = 64

vocabulary_size = 46210+1 #from Data Processing3.0
embedding_dim = 200

model = create_model(X_seq_len,output_dim,hidden_dim, vocabulary_size, embedding_dim)
name = 'second_RNN_even_data_more_embedding'

# Define a tensorboard callback
tb = TensorBoard(log_dir='./logs/'+name)
    
# Print model summary and train
model.summary()
current_history = model.fit(X_train_here_flat, Y_train_here, verbose=1, batch_size=b_size, epochs=num_epochs, callbacks=[tb], validation_split=0.2);

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 100, 200)          9242200   
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               120400    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 9,362,701
Trainable params: 9,362,701
Non-trainable params: 0
_________________________________________________________________
Train on 159586 samples, validate on 39897 samples
Epoch 1/10
159586/159586 [==============================] - 620s 4ms/step - loss: 0.4791 - acc: 0.7555 - val_loss: 0.4113 - val_acc: 0.8250
Epoch 2/10
159586/159586 [==============================] - 625s 4ms/step - loss: 0.3032 - acc: 0.8769 - val_loss: 0.2930 - val_acc: 0.8809
Epoch 3/10
159586/159586 [==============================] - 625s 

KeyboardInterrupt: 

### LSTM more hidden and embedding

In [16]:
X_seq_len = len(X_train_here[0]) #the input data is padded
output_dim = 1
hidden_dim = 256
num_epochs = 10
b_size = 518

vocabulary_size = 46210+1 #from Data Processing3.0
embedding_dim = 300

model = create_model(X_seq_len,output_dim,hidden_dim, vocabulary_size, embedding_dim)
name = 'second_RNN_LSTM_more_all'

# Define a tensorboard callback
tb = TensorBoard(log_dir='./logs/'+name)
    
# Print model summary and train
model.summary()
current_history = model.fit(X_train_here_flat, Y_train_here, verbose=1, batch_size=b_size, epochs=num_epochs, callbacks=[tb], validation_split=0.2);

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 100, 300)          13863300  
_________________________________________________________________
lstm_3 (LSTM)                (None, 256)               570368    
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 257       
Total params: 14,433,925
Trainable params: 14,433,925
Non-trainable params: 0
_________________________________________________________________
Train on 159586 samples, validate on 39897 samples
Epoch 1/10
159586/159586 [==============================] - 119s 747us/step - loss: 0.5021 - acc: 0.7314 - val_loss: 0.3633 - val_acc: 0.8557
Epoch 2/10
159586/159586 [==============================] - 118s 737us/step - loss: 0.2900 - acc: 0.8860 - val_loss: 0.2850 - val_acc: 0.8844
Epoch 3/10
159586/159586 [==============================] -

KeyboardInterrupt: 

### Save LSTM model

In [17]:
model.save(name+'.h5')

### GRU model

In [7]:
def create_model_GRU(X_seq_len, output_n, hidden_dim, vocabulary_size, embedding_dim):

    model = Sequential()
    model.add(Embedding(vocabulary_size, embedding_dim, input_length=X_seq_len, mask_zero=True))
    model.add(GRU(hidden_dim))
    model.add(Dense(output_n,activation='sigmoid'))
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [8]:
X_seq_len = len(X_train_here[0]) #the input data is padded
output_dim = 1
hidden_dim = 100
num_epochs = 10
b_size = 256

vocabulary_size = 46210+1 #from Data Processing3.0
embedding_dim = 200

model = create_model_GRU(X_seq_len,output_dim,hidden_dim, vocabulary_size, embedding_dim)
name = 'second_RNN_GRU'

# Define a tensorboard callback
tb = TensorBoard(log_dir='./logs/'+name)
    
# Print model summary and train
model.summary()
current_history = model.fit(X_train_here_flat, Y_train_here, verbose=1, batch_size=b_size, epochs=num_epochs, callbacks=[tb], validation_split=0.2);

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 200)          9242200   
_________________________________________________________________
gru_1 (GRU)                  (None, 100)               90300     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 9,332,601
Trainable params: 9,332,601
Non-trainable params: 0
_________________________________________________________________
Train on 159586 samples, validate on 39897 samples
Epoch 1/10
159586/159586 [==============================] - 151s 946us/step - loss: 0.3685 - acc: 0.8401 - val_loss: 0.2997 - val_acc: 0.8791
Epoch 2/10
159586/159586 [==============================] - 147s 920us/step - loss: 0.2488 - acc: 0.9029 - val_loss: 0.2731 - val_acc: 0.8909
Epoch 3/10
159586/159586 [==============================] - 1

### GRU model less param

In [11]:
X_seq_len = len(X_train_here[0]) #the input data is padded
output_dim = 1
hidden_dim = 50
num_epochs = 10
b_size = 256

vocabulary_size = 46210+1 #from Data Processing3.0
embedding_dim = 200

model = create_model_GRU(X_seq_len,output_dim,hidden_dim, vocabulary_size, embedding_dim)
name = 'second_RNN_GRU_less_hidden'

# Define a tensorboard callback
tb = TensorBoard(log_dir='./logs/'+name)
    
# Print model summary and train
model.summary()
current_history = model.fit(X_train_here_flat, Y_train_here, verbose=1, batch_size=b_size, epochs=num_epochs, callbacks=[tb], validation_split=0.2);

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 100, 200)          9242200   
_________________________________________________________________
gru_2 (GRU)                  (None, 50)                37650     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 51        
Total params: 9,279,901
Trainable params: 9,279,901
Non-trainable params: 0
_________________________________________________________________
Train on 159586 samples, validate on 39897 samples
Epoch 1/10
159586/159586 [==============================] - 144s 904us/step - loss: 0.3762 - acc: 0.8355 - val_loss: 0.3160 - val_acc: 0.8718
Epoch 2/10
159586/159586 [==============================] - 146s 914us/step - loss: 0.2555 - acc: 0.8996 - val_loss: 0.2741 - val_acc: 0.8909
Epoch 3/10
159586/159586 [==============================] - 1

KeyboardInterrupt: 

### GRU more params

In [13]:
X_seq_len = len(X_train_here[0]) #the input data is padded
output_dim = 1
hidden_dim = 256
num_epochs = 10
b_size = 256

vocabulary_size = 46210+1 #from Data Processing3.0
embedding_dim = 300

model = create_model_GRU(X_seq_len,output_dim,hidden_dim, vocabulary_size, embedding_dim)
name = 'second_RNN_GRU_more_all'

# Define a tensorboard callback
tb = TensorBoard(log_dir='./logs/'+name)
    
# Print model summary and train
model.summary()
current_history = model.fit(X_train_here_flat, Y_train_here, verbose=1, batch_size=b_size, epochs=num_epochs, callbacks=[tb], validation_split=0.2);

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 100, 300)          13863300  
_________________________________________________________________
gru_4 (GRU)                  (None, 256)               427776    
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 257       
Total params: 14,291,333
Trainable params: 14,291,333
Non-trainable params: 0
_________________________________________________________________
Train on 159586 samples, validate on 39897 samples
Epoch 1/10
159586/159586 [==============================] - 157s 982us/step - loss: 0.3687 - acc: 0.8383 - val_loss: 0.2908 - val_acc: 0.8817
Epoch 2/10
159586/159586 [==============================] - 157s 984us/step - loss: 0.2402 - acc: 0.9059 - val_loss: 0.2622 - val_acc: 0.8962
Epoch 3/10
159586/159586 [==============================] -

KeyboardInterrupt: 

### Save GRU model

In [14]:
model.save(name+'.h5')

### Regularized GRU

In [ ]:
def create_model_GRU_2layer(X_seq_len, output_n, hidden_dim, vocabulary_size, embedding_dim):

    model = Sequential()
    model.add(Embedding(vocabulary_size, embedding_dim, input_length=X_seq_len, mask_zero=True))
    model.add(GRU(hidden_dim,kernel_regularizer=regularizers.l2(0.01)))
    model.add(Dense(output_n,activation='sigmoid'))
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [ ]:
X_seq_len = len(X_train_here[0]) #the input data is padded
output_dim = 1
hidden_dim = 256
num_epochs = 10
b_size = 256

vocabulary_size = 46210+1 #from Data Processing3.0
embedding_dim = 300

model = create_model_GRU(X_seq_len,output_dim,hidden_dim, vocabulary_size, embedding_dim)
name = 'second_RNN_GRU_regularized'

# Define a tensorboard callback
tb = TensorBoard(log_dir='./logs/'+name)
    
# Print model summary and train
model.summary()
current_history = model.fit(X_train_here_flat, Y_train_here, verbose=1, batch_size=b_size, epochs=num_epochs, callbacks=[tb], validation_split=0.2);

### GRU model with two layers

In [ ]:
def create_model_GRU_2layer(X_seq_len, output_n, hidden_dim, vocabulary_size, embedding_dim):

    model = Sequential()
    model.add(Embedding(vocabulary_size, embedding_dim, input_length=X_seq_len, mask_zero=True))
    model.add(GRU(hidden_dim))
    model.add(GRU(hidden_dim))
    model.add(Dense(output_n,activation='sigmoid'))
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [ ]:
X_seq_len = len(X_train_here[0]) #the input data is padded
output_dim = 1
hidden_dim = 50
num_epochs = 10
b_size = 256

vocabulary_size = 46210+1 #from Data Processing3.0
embedding_dim = 200

model = create_model_GRU(X_seq_len,output_dim,hidden_dim, vocabulary_size, embedding_dim)
name = 'second_RNN_GRU_less_hidden'

# Define a tensorboard callback
tb = TensorBoard(log_dir='./logs/'+name)
    
# Print model summary and train
model.summary()
current_history = model.fit(X_train_here_flat, Y_train_here, verbose=1, batch_size=b_size, epochs=num_epochs, callbacks=[tb], validation_split=0.2);

### Attention model

In [9]:
def attention_3d_block(inputs):
    # inputs.shape = (batch_size, time_steps, input_dim)
    input_dim = int(inputs.shape[2])
    a = Permute((2, 1))(inputs)
    a = Reshape((input_dim, X_seq_len))(a) # this line is not useful. It's just to know which dimension is what.
    a = Dense(X_seq_len, activation='softmax')(a)
    if SINGLE_ATTENTION_VECTOR:
        a = Lambda(lambda x: K.mean(x, axis=1), name='dim_reduction')(a)
        a = RepeatVector(input_dim)(a)
    a_probs = Permute((2, 1), name='attention_vec')(a)
    #output_attention_mul = merge([inputs, a_probs], name='attention_mul', mode='mul')
    output_attention_mul = multiply([inputs, a_probs])
    return output_attention_mul

def create_model_attention(X_seq_len, output_n, hidden_dim, vocabulary_size, embedding_dim):
        
    INPUTS = Input(shape=[X_seq_len])
    embedded_inputs = Embedding(vocabulary_size, embedding_dim, input_length=X_seq_len)(INPUTS)
    
    #def model_attention_applied_before_lstm():
    attention_mul = attention_3d_block(embedded_inputs)
    lstm_units = hidden_dim
    attention_mul = LSTM(lstm_units, return_sequences=False)(attention_mul)
    output = Dense(1, activation='sigmoid')(attention_mul)
    model = Model(input=[INPUTS], output=output)
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [10]:
X_seq_len = len(X_train_here[0]) #the input data is padded
output_dim = 1
hidden_dim = 20
num_epochs = 10
b_size = 256
# if True, the attention vector is shared across the input_dimensions where the attention is applied.
SINGLE_ATTENTION_VECTOR = False

vocabulary_size = 46210+1 #from Data Processing3.0
embedding_dim = 200

model = create_model_attention(X_seq_len,output_dim,hidden_dim, vocabulary_size, embedding_dim)
name = 'second_RNN_attention'

# Define a tensorboard callback
tb = TensorBoard(log_dir='./logs/'+name)
    
# Print model summary and train
model.summary()
current_history = model.fit(X_train_here_flat, Y_train_here, verbose=1, batch_size=b_size, epochs=num_epochs, callbacks=[tb], validation_split=0.2);

/home/student/anaconda3/envs/dml_gpu/lib/python3.6/site-packages/ipykernel_launcher.py:25: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 100, 200)     9242200     input_1[0][0]                    
__________________________________________________________________________________________________
permute_1 (Permute)             (None, 200, 100)     0           embedding_2[0][0]                
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 200, 100)     0           permute_1[0][0]                  
__________________________________________________________________________________________________
dense_2 (D